## **TML Assignment 4 - Task 2**

## **TEAM 16**

## **Ahrar Bin Aslam and Muhammad Mubeen Siddiqui**

## **Setting Up the Environment and Downloading the Images**

In [2]:
import os
import requests

# Create output directory
os.makedirs("imagenet_images", exist_ok=True)

# ImageNet sample image URLs
image_urls = {
    "West_Highland_white_terrier": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n02098286_West_Highland_white_terrier.JPEG",
    "American_coot": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n02018207_American_coot.JPEG",
    "racer": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n04037443_racer.JPEG",
    "flamingo": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n02007558_flamingo.JPEG",
    "kite": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n01608432_kite.JPEG",
    "goldfish": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n01443537_goldfish.JPEG",
    "tiger_shark": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n01491361_tiger_shark.JPEG",
    "vulture": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n01616318_vulture.JPEG",
    "common_iguana": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n01677366_common_iguana.JPEG",
    "orange": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n07747607_orange.JPEG",
}

# Download images
for name, url in image_urls.items():
    filename = f"imagenet_images/{name}.jpeg"
    if not os.path.exists(filename):
        print(f"Downloading {name}...")
        response = requests.get(url)
        with open(filename, 'wb') as f:
            f.write(response.content)
print("All images downloaded successfully.")


All images downloaded successfully.


## **Installing Grad-CAM**

In [3]:
!pip install grad-cam -q
from pytorch_grad_cam import GradCAM
print("Grad-CAM installed successfully")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.0 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.

## **Installing Dependencies**

In [4]:
from PIL import Image
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from pytorch_grad_cam import GradCAM, ScoreCAM, AblationCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

## **Load and Prepare Pretrained ResNet50 Model for Inference**

In [5]:
model = models.resnet50(pretrained=True)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 196MB/s] 


## **Grad-CAM, AblationCAM, and ScoreCAM**

In [6]:
output_dir = "cam_outputs"
os.makedirs(output_dir, exist_ok=True)

# Last conv layer for ResNet50
target_layer = model.layer4[-1]

# Loop through each image
for name in image_urls:
    img_path = f"imagenet_images/{name}.jpeg"
    image = Image.open(img_path).convert('RGB')

    # Preprocess image
    input_tensor = transform(image).unsqueeze(0).to(device)
    resized_image = transforms.Resize((224, 224))(image)
    rgb_image_np = transforms.ToTensor()(resized_image).permute(1, 2, 0).numpy()
    rgb_image_np = rgb_image_np / rgb_image_np.max()

    # Predict class
    with torch.no_grad():
        output = model(input_tensor)
        class_idx = output.argmax().item()
    targets = [ClassifierOutputTarget(class_idx)]

    # Grad-CAM
    grad_cam = GradCAM(model=model, target_layers=[target_layer])
    cam_map = grad_cam(input_tensor=input_tensor, targets=targets)[0]
    gradcam_vis = show_cam_on_image(rgb_image_np, cam_map, use_rgb=True)
    del grad_cam

    # AblationCAM
    ablation_cam = AblationCAM(model=model, target_layers=[target_layer])
    cam_map = ablation_cam(input_tensor=input_tensor, targets=targets)[0]
    ablation_vis = show_cam_on_image(rgb_image_np, cam_map, use_rgb=True)
    del ablation_cam

    # ScoreCAM
    score_cam = ScoreCAM(model=model, target_layers=[target_layer])
    cam_map = score_cam(input_tensor=input_tensor, targets=targets)[0]
    score_vis = show_cam_on_image(rgb_image_np, cam_map, use_rgb=True)
    del score_cam

    # Plot and save
    plt.figure(figsize=(12, 4))
    plt.suptitle(f"{name}", fontsize=14)

    plt.subplot(1, 4, 1)
    plt.imshow(image)
    plt.axis('off')
    plt.title("Original")

    plt.subplot(1, 4, 2)
    plt.imshow(gradcam_vis)
    plt.axis('off')
    plt.title("Grad-CAM")

    plt.subplot(1, 4, 3)
    plt.imshow(ablation_vis)
    plt.axis('off')
    plt.title("AblationCAM")

    plt.subplot(1, 4, 4)
    plt.imshow(score_vis)
    plt.axis('off')
    plt.title("ScoreCAM")

    plt.tight_layout()
    plt.savefig(f"{output_dir}/{name}_cams.png")
    plt.close()
    print(f"Saved: {output_dir}/{name}_cams.png")

100%|██████████| 128/128 [00:06<00:00, 20.40it/s]


Saved: cam_outputs/West_Highland_white_terrier_cams.png


100%|██████████| 128/128 [00:05<00:00, 21.40it/s]


Saved: cam_outputs/American_coot_cams.png


100%|██████████| 128/128 [00:06<00:00, 20.93it/s]


...

Saved: cam_outputs/vulture_cams.png


100%|██████████| 128/128 [00:07<00:00, 17.96it/s]


Saved: cam_outputs/common_iguana_cams.png


100%|██████████| 128/128 [00:06<00:00, 18.78it/s]


Saved: cam_outputs/orange_cams.png
